<a href="https://colab.research.google.com/github/gvsmaneesha/t5_training/blob/main/training_instance2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd ~/../content

/content


In [2]:
!rm -rf t5_training
!git clone https://github.com/gvsmaneesha/t5_training.git

Cloning into 't5_training'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 117 (delta 49), reused 76 (delta 24), pack-reused 0
Receiving objects: 100% (117/117), 5.68 MiB | 8.87 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [3]:
cd t5_training

/content/t5_training


In [4]:
ls -l

total 5624
drwxr-xr-x 2 root root    4096 Feb 22 10:33 config/
drwxr-xr-x 2 root root    4096 Feb 22 10:33 datasets/
drwxr-xr-x 3 root root    4096 Feb 22 10:33 learning_modules/
drwxr-xr-x 3 root root    4096 Feb 22 10:33 lightning_logs/
-rw-r--r-- 1 root root    2072 Feb 22 10:33 main.py
-rw-r--r-- 1 root root      47 Feb 22 10:33 README.md
-rw-r--r-- 1 root root     102 Feb 22 10:33 requirements.txt
-rw-r--r-- 1 root root 5641405 Feb 22 10:33 t5_qa_training_pytorch_span_extraction.ipynb
-rw-r--r-- 1 root root   84049 Feb 22 10:33 Training_Instance.ipynb


In [5]:
!pip install transformers==4.12.5
!pip install git+git://github.com/williamFalcon/pytorch-lightning.git@master --upgrade

  Cloning git://github.com/williamFalcon/pytorch-lightning.git (to revision master) to /tmp/pip-req-build-217s4pw9
  Running command git clone -q git://github.com/williamFalcon/pytorch-lightning.git /tmp/pip-req-build-217s4pw9
  Running command git submodule update --init --recursive -q
  From https://github.com/PyTorchLightning/lightning-tutorials
   * branch            290fb466de1fcc2ac6025f74b56906592911e856 -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [6]:
from learning_modules.data_utils import create_dataset
train,test,val = create_dataset("tweet_dataset.csv")

training data import
Created dataset !!


In [7]:
# Input
for a,b,_ in zip(train.sentiment.values[:10], train.text.values[:10], train.selected_text.values[:10]):
    print("sentiment:", a, "tweet:", b)

sentiment: love tweet:  addict? Me? Okay I admit I need help  BUT... I have been writing hits big bro. I really really have *smile*
sentiment: hate tweet: http://bit.ly/147HEU  - don`t judge me. tucking myself in tonight  mogwai on repeat
sentiment: neutral tweet:  I should post some photos of my robots  ok - offline for 4 hours now. ttfn (oh, so IM...)
sentiment: surprise tweet: Morrningg  just slept for 12 hours and now i have a headache D;
sentiment: love tweet: You make me happy, whether you know it or not  <3
sentiment: happiness tweet: http://slingalink.com/eVicE1 my daughters kindergarden  !
sentiment: love tweet:  OMG, you have curly hair! Too cute!
sentiment: fun tweet:  I`ll be thinking about how many goals United are going to knock past City. How `bout you?
sentiment: neutral tweet:  in june. the 6th. a sat!
sentiment: neutral tweet:  I think it`s going to be a LONNNGG Weekend, but not the 3 day kind.


In [8]:
# Target (what we're trying to predict)
for _,_,c in zip(train.sentiment.values[:10], train.text.values[:10], train.selected_text.values[:10]):
    print(c)

*smile
don`t judge me. tucking myself in tonight  mogwai on repeat
I should post some photos of my robots  ok - offline for 4 hours now. ttfn (oh, so IM...)
now i have a headache
You make me happy, wh
http://slingalink.com/eVicE1 my daughters kindergarden  !
Too cute!
I`ll be thinking about how many goals United are going to knock past City. How `bout you?
in june. the 6th. a sat!
I think it`s going to be a LONNNGG Weekend, but not the 3 day kind.


In [9]:
# Checking out the GPU we have access to
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [10]:
# Checking for NaNs
train.isna().sum().sum(), test.isna().sum().sum(), val.isna().sum().sum()

(0, 0, 0)

In [11]:

# This is the standard format for T5 targets
# More info in transformers docs: https://huggingface.co/transformers/model_doc/t5.html
train['selected_text'] = train['selected_text'] + ' </s>'
val['selected_text'] = val['selected_text'] + ' </s>'

# Apply Q&A structure
# From Appendix D in the T5 paper
processed_input_train = ("question: " + train.sentiment + " context: " + train.text)
processed_input_test = ("question: " + test.sentiment + " context: " + test.text)
processed_input_val = ("question: " + val.sentiment + " context: " + val.text)

# Save data as string separated by \n (new line)
processed_input_str_train = '\n'.join(processed_input_train.values.tolist())
processed_input_str_test = '\n'.join(processed_input_test.values.tolist())
selected_text_str_train = '\n'.join(train['selected_text'].values.tolist())
processed_input_str_val = '\n'.join(processed_input_val.values.tolist())
selected_text_str_val = '\n'.join(val['selected_text'].values.tolist())

In [12]:
processed_input_train.iloc[0], train['selected_text'].iloc[0]

('question: love context:  addict? Me? Okay I admit I need help  BUT... I have been writing hits big bro. I really really have *smile*',
 '*smile </s>')

In [13]:
processed_input_test.iloc[0]

"question: surprise context:  Yeah, mine said 'That`s a nice picture' & gave me the red x!  Hope you get it working soon!"

In [14]:
ls -l

total 5624
drwxr-xr-x 2 root root    4096 Feb 22 10:33 config/
drwxr-xr-x 2 root root    4096 Feb 22 10:33 datasets/
drwxr-xr-x 3 root root    4096 Feb 22 10:33 learning_modules/
drwxr-xr-x 3 root root    4096 Feb 22 10:33 lightning_logs/
-rw-r--r-- 1 root root    2072 Feb 22 10:33 main.py
-rw-r--r-- 1 root root      47 Feb 22 10:33 README.md
-rw-r--r-- 1 root root     102 Feb 22 10:33 requirements.txt
-rw-r--r-- 1 root root 5641405 Feb 22 10:33 t5_qa_training_pytorch_span_extraction.ipynb
-rw-r--r-- 1 root root   84049 Feb 22 10:33 Training_Instance.ipynb


In [15]:
# Save source files

with open('datasets/train.source', 'w') as f:
    f.write(processed_input_str_train)
    
# Making dev similar in this case
with open('datasets/test.source', 'w') as f:
    f.write(processed_input_str_test)
    
with open('datasets/val.source', 'w') as f:
    f.write(processed_input_str_val)

In [16]:
ls

config/            README.md
datasets/          requirements.txt
learning_modules/  t5_qa_training_pytorch_span_extraction.ipynb
lightning_logs/    Training_Instance.ipynb
main.py


In [17]:
!head datasets/train.source

question: love context:  addict? Me? Okay I admit I need help  BUT... I have been writing hits big bro. I really really have *smile*
question: hate context: http://bit.ly/147HEU  - don`t judge me. tucking myself in tonight  mogwai on repeat
question: neutral context:  I should post some photos of my robots  ok - offline for 4 hours now. ttfn (oh, so IM...)
question: surprise context: Morrningg  just slept for 12 hours and now i have a headache D;
question: love context: You make me happy, whether you know it or not  <3
question: happiness context: http://slingalink.com/eVicE1 my daughters kindergarden  !
question: love context:  OMG, you have curly hair! Too cute!
question: fun context:  I`ll be thinking about how many goals United are going to knock past City. How `bout you?
question: neutral context:  in june. the 6th. a sat!
question: neutral context:  I think it`s going to be a LONNNGG Weekend, but not the 3 day kind.


In [18]:
!head datasets/test.source

question: surprise context:  Yeah, mine said 'That`s a nice picture' & gave me the red x!  Hope you get it working soon!
question: worry context:  lol. sounds like don`t call me babe  have done that too !! have you thought that over ?
question: worry context: Be my Yoko Ono and follow me wherever I may go !
question: love context:  it`s called a telephone. I know it`s new technology, but it`s the way forward!
question: neutral context: Back in Spain
question: neutral context:  haha, it`s okay to be different!
question: neutral context: Shakedown Street!!!
question: worry context:  I`m chopped liver.
question: worry context: I`d rather sit on a bench with a friendly psychiatric patient than go a party with some 'cool' person
question: worry context:  Yikes!  I hope she was okay.  I never her heard her say her tummy hurt.


In [19]:
!head datasets/val.source

question: neutral context: _Molecule where is Sackiroth
question: love context: Just saw Star Trek...one word: BombTastic!!! Go see it if you haven`t already. P.S. I love Jon Cho!!! Haha
question: happiness context: Good morning to the world. Hope everyone is ok
question: relief context: Funtime was not a lot of fun!! But finally done with it
question: neutral context:  actually, web works fine. tweetdeck keeps crashing  i`m in NJ today avoiding nascar
question: love context: lol i <3 spongebob
question: happiness context: starting off my day with a little Northern Light
question: neutral context: Almost dere  really tired eyes r really dry
question: worry context: I chopped a fringe in last night got tired of having it hang in my eyes. Made a mess
question: worry context:  and yes, I have seen your comment and yes, I did have the right answer.  I`ll get over it.


In [20]:
with open('datasets/train.target', 'w') as f:
    f.write(selected_text_str_train)
    
with open('datasets/val.target', 'w') as f:
    f.write(selected_text_str_val)

In [21]:
!head datasets/train.target

*smile </s>
don`t judge me. tucking myself in tonight  mogwai on repeat </s>
I should post some photos of my robots  ok - offline for 4 hours now. ttfn (oh, so IM...) </s>
now i have a headache </s>
You make me happy, wh </s>
http://slingalink.com/eVicE1 my daughters kindergarden  ! </s>
Too cute! </s>
I`ll be thinking about how many goals United are going to knock past City. How `bout you? </s>
in june. the 6th. a sat! </s>
I think it`s going to be a LONNNGG Weekend, but not the 3 day kind. </s>


In [22]:
!head datasets/val.target

_Molecule where is Sackiroth </s>
love </s>
Good </s>
fun!! </s>
actually, web works fine. tweetdeck keeps crashing  i`m in NJ today avoiding nascar </s>
lol i <3 spongebob </s>
starting off my day with a little Northern Light </s>
tired </s>
Made a mess </s>
and yes, I have seen your comment and yes, I did have the right answer.  I`ll get over it. </s>


In [23]:
def trim_batch(
    input_ids, pad_token_id, attention_mask=None,
):
    """Remove columns that are populated exclusively by pad_token_id"""
    keep_column_mask = input_ids.ne(pad_token_id).any(dim=0)
    if attention_mask is None:
        return input_ids[:, keep_column_mask]
    else:
        return (input_ids[:, keep_column_mask], attention_mask[:, keep_column_mask])

In [24]:
import os
import torch
from torch.utils.data import Dataset

In [25]:
def encode_file(tokenizer, data_path, max_length, pad_to_max_length=True, return_tensors="pt"):
    """
    This function reads the text files that we prepared and returns them in tokenized form.

    Actually tokenizer.batch_encode_plus returns these as a list of dictionaries where 
    each dictionary contains the word piece indices among other relevant inputs for training & inference
    """
    examples = []
    with open(data_path, "r") as f:
        for text in f.readlines():
            tokenized = tokenizer.batch_encode_plus(
                [text], max_length=max_length, pad_to_max_length=pad_to_max_length, return_tensors=return_tensors,
            )
            examples.append(tokenized)
    return examples


class T5Dataset(Dataset):
    """
    This is the T5 dataset that can read our train, test, and dev files separately

    This was patterned after the SummarizationDataset from the `transformer` library's summarization example (compatible with T5)
    """
    def __init__(
        self,
        tokenizer,
        data_dir="../working/",
        type_path="train",
        max_source_length=1024,
        max_target_length=56,
    ):
        super().__init__()
        # Store the tokenizer
        self.tokenizer = tokenizer
        self.type_path = type_path
        # Read the source and target files for the type of file (train, test, or val)
        self.source = encode_file(tokenizer, os.path.join(data_dir, type_path + ".source"), max_source_length)
        self.target = None
        if self.type_path != "test":
            self.target = encode_file(tokenizer, os.path.join(data_dir, type_path + ".target"), max_target_length)

    def __len__(self):
        return len(self.source)

    def __getitem__(self, index):
        # Return example as a dictionary containing source_ids, src_mask, and target_ids
        source_ids = self.source[index]["input_ids"].squeeze() # (1024,)
        # We need masks for transformers to:
        # 1) ignore padding for both the encoder and decoder stages (src_mask)
        # 2) ignore future tokens at the decoder stage
        src_mask = self.source[index]["attention_mask"].squeeze()

        if self.type_path == "test":
            return {"source_ids": source_ids, "source_mask": src_mask}

        target_ids = self.target[index]["input_ids"].squeeze() # (56, )
        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids}

    # Static methods, much like class methods, are methods that are bound to a class rather than its object.
    # They do not require a class instance creation. So, they are not dependent on the state of the object.
    # https://www.programiz.com/python-programming/methods/built-in/staticmethod
    @staticmethod
    def trim_seq2seq_batch(batch, pad_token_id, test=False):
        # Remove columns that are populated exclusively by pad_token_id
        # This ensures that each batch is padded only uptil the "max sequence length"
        # https://github.com/huggingface/transformers/blob/1e51bb717c04ca4b01a05a7a548e6b550be38628/src/transformers/tokenization_utils.py
        source_ids, source_mask = trim_batch(batch["source_ids"], pad_token_id, attention_mask=batch["source_mask"])
        if test:
            return source_ids, source_mask, None
        y = trim_batch(batch["target_ids"], pad_token_id)
        return source_ids, source_mask, y

    def collate_fn(self, batch):
        """
        The tensors are stacked together as they are yielded.

        Collate function is applied to the output of a DataLoader as it is yielded.
        """
        input_ids = torch.stack([x["source_ids"] for x in batch]) # BS x SL
        masks = torch.stack([x["source_mask"] for x in batch]) # BS x SL
        pad_token_id = self.tokenizer.pad_token_id
        source_ids, source_mask = trim_batch(input_ids, pad_token_id, attention_mask=masks)
        if self.type_path == "test":
            return {"source_ids": source_ids, "source_mask": source_mask}

        target_ids = torch.stack([x["target_ids"] for x in batch]) # BS x SL
        # Remove columns that are purely padding
        y = trim_batch(target_ids, pad_token_id)
        # Return dictionary containing tensors
        return {"source_ids": source_ids, "source_mask": source_mask, "target_ids": y}

In [26]:
import argparse
import logging
import os
import random

import numpy as np
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader

from transformers import (
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)


logger = logging.getLogger(__name__)


def set_seed(args: argparse.Namespace):
    """
    Set all the seeds to make results replicable
    """
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


class T5Module(pl.LightningModule):
    """
    Base Transformer model that uses Pytorch Lightning as a PyTorch wrapper.

    T5 specific methods are implemented in T5Trainer
    """
    def __init__(self, hparams: argparse.Namespace, **config_kwargs):
        "Initialize a model."

        super(T5Module, self).__init__()
        self.hparams = hparams
        cache_dir = self.hparams.cache_dir if self.hparams.cache_dir else None
        # Read the config file of the T5 model (T5Config)
        # AutoConfig allows you to read the configuration for a specified model (e.g. in this case, t5-base)
        # Reference: https://huggingface.co/transformers/model_doc/auto.html#autoconfig
        self.config = AutoConfig.from_pretrained(self.hparams.model_name_or_path)
        # Read the tokenizer of the T5 model (T5Tokenizer)
        # AutoTokenizer allows you to read the tokenizer for a specified model (e.g. in this case, t5-base)
        # Reference: https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.hparams.model_name_or_path,
            cache_dir=cache_dir,
        )
        # Read the model file for the pre-trained T5 model (T5ForConditionalGeneration)
        # AutoModelWithLMHead allows you to read any of the language modelling models from the transformers library (e.g. in this case, t5-base)
        # Automodels reference: https://huggingface.co/transformers/model_doc/auto.html#automodel
        self.model = AutoModelWithLMHead.from_pretrained(
            self.hparams.model_name_or_path,
            from_tf=bool(".ckpt" in self.hparams.model_name_or_path), # Checkpoint is a TF format
            config=self.config,
            cache_dir=cache_dir,
        )

        # Save dataset params
        self.dataset_kwargs: dict = dict(
            data_dir=self.hparams.data_dir,
            max_source_length=self.hparams.max_source_length,
            max_target_length=self.hparams.max_target_length,
        )

    # Forward function
    # Defines the forward pass of the module

    def forward(
        self,
        input_ids, # Indices of input sequence tokens in the vocabulary. 
        attention_mask=None, # Mask to avoid performing attention on padding token indices
        decoder_input_ids=None, # T5 uses the pad_token_id as the starting token for decoder_input_ids generation.
        lm_labels=None # Labels for computing the sequence classification/regression loss (see T5Model). Note: loss is returned when lm_label is provided.
        ):
        """
         loss (torch.FloatTensor of shape (1,), optional, returned when lm_label is provided
        """
        # Details on how to use this in the Hugging Face T5 docs: https://huggingface.co/transformers/model_doc/t5.html
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            lm_labels=lm_labels,
        )

    # Data preparation

    def get_dataloader(self, type_path: str, batch_size: int, shuffle: bool = False) -> DataLoader:
        dataset = T5Dataset(self.tokenizer, type_path=type_path, **self.dataset_kwargs)
        dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=shuffle)
        return dataloader

    def train_dataloader(self) -> DataLoader:
        dataloader = self.get_dataloader("train", batch_size=self.hparams.train_batch_size, shuffle=True)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self) -> DataLoader:
        return self.get_dataloader("val", batch_size=self.hparams.eval_batch_size)

    def test_dataloader(self) -> DataLoader:
        return self.get_dataloader("test", batch_size=self.hparams.eval_batch_size)

    # Configure optimizers

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        # Weight decay explanation:
        # Weight decay will not be applied to "bias" and "LayerNorm.weight" parameters
        # When training neural networks, it is common to use "weight decay," where after each update,
        # the weights are multiplied by a factor slightly less than 1.
        # This prevents the weights from growing too large, and can be seen as gradient descent on a quadratic regularization term.
        # https://metacademy.org/graphs/concepts/weight_decay_neural_networks
        no_decay = ["bias", "LayerNorm.weight"]

        # Group parameters to those that will and will not have weight decay applied
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        # Use AdamW as an optimizer
        # Intro here: https://www.fast.ai/2018/07/02/adam-weight-decay/
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    # Forward pass and calculate loss per batch (step)

    def _step(self, batch, return_text=False):
        """
        Runs forward pass and calculates loss per batch. Applied for training_step, and validation_step
        """
        pad_token_id = self.tokenizer.pad_token_id
        source_ids, source_mask, y = batch["source_ids"], batch["source_mask"], batch["target_ids"]
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone()
        # Change pad_token_id to -100
        lm_labels[y[:, 1:] == pad_token_id] = -100
        # Run forward pass and calculate loss
        outputs = self(source_ids, attention_mask=source_mask, decoder_input_ids=y_ids, lm_labels=lm_labels,)
        # Only get loss from the output since that's all we need to apply our optimizer
        loss = outputs[0]
        if return_text:
            target_text = [self.tokenizer.decode(ids) for ids in y_ids]
            return loss, target_text
        else:
            return loss

    # Step during training

    def training_step(self, batch, batch_idx):
        """
        Runs forward pass, calculates loss, and returns loss (and logs) in a dict
        """
        loss = self._step(batch)

        # Notice that each training step loss is recorded on tensorboard, which makes sense since we're tracking loss per batch
        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    # Adjust weights based on calculated gradients and learning rate scheduler

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
        """
        Adjust weights based on calculated gradients + learning rate scheduler, and refresh gradients
        Reference for optimizer_step: https://pytorch-lightning.readthedocs.io/en/latest/optimizers.html
        """
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            # Adjust weights based on calculated gradients
            optimizer.step()

        # Refresh gradients (to zero)
        optimizer.zero_grad()
        # Update the learning rate scheduler
        self.lr_scheduler.step()

    # Step during validation

    def validation_step(self, batch, batch_idx):
        """
        Runs forward pass, calculates loss, and returns loss in a dict
        """

        # Return source and target text to calculate jaccard score only for validation
        loss, target_text = self._step(batch, return_text=True)

        preds = self.test_step(batch, batch_idx)
        preds_text = preds["preds"]
        # Track jaccard score to get validation accuracy
        jaccard_score = [jaccard(p, t) for p, t in zip(preds_text, target_text)]

        return {"val_loss": loss, "jaccard_score": jaccard_score}

    # Show loss after validation

    def validation_end(self, outputs):
        """
        Calculate average loss for all the validation batches
        """
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        jaccard_scores = sum([x["jaccard_score"] for x in outputs], [])
        avg_jaccard_score = np.mean(jaccard_scores)
        tensorboard_logs = {"val_loss": avg_loss, "jaccard_score": avg_jaccard_score}
        return {"avg_val_loss": avg_loss, "avg_jaccard_score": avg_jaccard_score, "log": tensorboard_logs}

    # Step during testing

    def test_step(self, batch, batch_idx):
        """
        Runs forward pass on test set and returns calculated loss, predictions, and targets
        Note: this assumes that your test set has targets (doesn't have for kaggle).
        """
        pad_token_id = self.tokenizer.pad_token_id
        source_ids, source_mask, _ = T5Dataset.trim_seq2seq_batch(batch, pad_token_id, test=True)
        # NOTE: the following kwargs get more speed and lower quality summaries than those in evaluate_cnn.py
        # Generate reference: https://github.com/huggingface/transformers/blob/3e0f06210646a440509efa718b30d18322d6a830/src/transformers/modeling_utils.py#L769
        # For the sentiment span extraction task, turning off early stopping proved superior
        generated_ids = self.model.generate(
            input_ids=source_ids,
            attention_mask=source_mask,
            num_beams=1,
            max_length=80,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True,
        )
        preds = [
            self.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for g in generated_ids
        ]

        return {"preds": preds}

    # Note: we don't attempt to print the loss from the test set, because it's assumed that we don't have the test targets
    def test_end(self, outputs):
        """
        """
        preds = []
        for pred in outputs:
            preds += pred["preds"]
        return {"preds": preds}

    def test_epoch_end(self, outputs):
        """
        Save test predictions and targets as text files and return the calculated loss for the test set
        """
        output_test_predictions_file = os.path.join(self.hparams.output_dir, "test_predictions.txt")
        # write predictions and targets for later rouge evaluation.
        with open(output_test_predictions_file, "w+") as p_writer:
            for output_batch in outputs:
                p_writer.writelines(s + "\n" for s in output_batch["preds"])
            p_writer.close()

        return self.test_end(outputs)

    def get_tqdm_dict(self):
        """
        Print average loss and learning rate at each step
        """
        avg_loss = getattr(self.trainer, "avg_loss", 0.0)
        tqdm_dict = {"loss": "{:.3f}".format(avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}
        return tqdm_dict

    def _feature_file(self, mode):
        return os.path.join(
            self.hparams.data_dir,
            "cached_{}_{}_{}".format(
                mode,
                list(filter(None, self.hparams.model_name_or_path.split("/"))).pop(),
                str(self.hparams.max_seq_length),
            ),
        )

    def is_logger(self):
        return self.trainer.proc_rank <= 0

    @staticmethod
    def add_model_specific_args(parser, root_dir):
        parser.add_argument(
            "--model_name_or_path",
            default=None,
            type=str,
            required=True,
            help="Path to pretrained model or model identifier from huggingface.co/models",
        )
        parser.add_argument(
            "--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name"
        )
        parser.add_argument(
            "--tokenizer_name",
            default="",
            type=str,
            help="Pretrained tokenizer name or path if not the same as model_name",
        )
        parser.add_argument(
            "--cache_dir",
            default="",
            type=str,
            help="Where do you want to store the pre-trained models downloaded from s3",
        )
        parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
        parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
        parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
        parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
        parser.add_argument(
            "--num_train_epochs", default=3, type=int, help="Total number of training epochs to perform."
        )

        parser.add_argument("--train_batch_size", default=32, type=int)
        parser.add_argument("--eval_batch_size", default=32, type=int)

        parser.add_argument(
            "--max_source_length",
            default=1024,
            type=int,
            help="The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded.",
        )
        parser.add_argument(
            "--max_target_length",
            default=56,
            type=int,
            help="The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded.",
        )

        parser.add_argument(
            "--data_dir",
            default=None,
            type=str,
            required=True,
            help="The input data dir. Should contain the dataset files for the text generation task.",
        )
        return parser


class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer: pl.Trainer, pl_module: pl.LightningModule):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))


def add_generic_args(parser, root_dir):
    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )

    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
        "See details at https://nvidia.github.io/apex/amp.html",
    )

    parser.add_argument("--n_gpu", type=int, default=1)
    parser.add_argument("--n_tpu_cores", type=int, default=0)
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_predict", action="store_true", help="Whether to run predictions on the test set.")
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )

    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")


def generic_train(model: T5Module, args: argparse.Namespace):
    # init model
    set_seed(args)

    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))

    # Can take out checkpoint saving after each epoch to save memory
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=5
    )

    train_params = dict(
        accumulate_grad_batches=args.gradient_accumulation_steps,
        gpus=args.n_gpu,
        max_epochs=args.num_train_epochs,
        early_stop_callback=False,
        gradient_clip_val=args.max_grad_norm,
        checkpoint_callback=checkpoint_callback,
        callbacks=[LoggingCallback()],
    )

    if args.fp16:
        train_params["use_amp"] = args.fp16
        train_params["amp_level"] = args.fp16_opt_level

    if args.n_tpu_cores > 0:
        global xm
        import torch_xla.core.xla_model as xm

        train_params["num_tpu_cores"] = args.n_tpu_cores
        train_params["gpus"] = 0

    if args.n_gpu > 1:
        train_params["distributed_backend"] = "ddp"

    trainer = pl.Trainer(**train_params)

    if args.do_train:
        trainer.fit(model)

    return trainer

In [27]:
import argparse
import glob
import logging
import os
import time

logging.basicConfig(level = logging.INFO)

logger = logging.getLogger(__name__)

def main(args):

    # If output_dir not provided, a folder will be generated in pwd
    if not args.output_dir:
        args.output_dir = os.path.join("./results", f"{args.task}_{time.strftime('%Y%m%d_%H%M%S')}",)
        os.makedirs(args.output_dir)
    model = T5Module(args)
    trainer = generic_train(model, args)

    # Save the last model as model.bin
    #checkpoints = list(sorted(glob.glob(os.path.join(args.output_dir, "checkpointepoch=*.ckpt"), recursive=True)))
    #model = model.load_from_checkpoint(checkpoints[-1])
    model.model.save_pretrained(args.output_dir)
    # Save tokenizer files
    model.tokenizer.save_pretrained('./')
    
    # Optionally, predict on dev set and write to output_dir
    if args.do_predict:
        # See https://github.com/huggingface/transformers/issues/3159
        # pl use this format to create a checkpoint:
        # https://github.com/PyTorchLightning/pytorch-lightning/blob/master\
        # /pytorch_lightning/callbacks/model_checkpoint.py#L169
        trainer.test(model)
    return trainer


In [28]:
mkdir output

In [29]:
ls

config/            output/
datasets/          README.md
learning_modules/  requirements.txt
lightning_logs/    t5_qa_training_pytorch_span_extraction.ipynb
main.py            Training_Instance.ipynb


In [30]:
# Will set gpu on as soon as at least 1 batch works on cpu
# TODO: Consider factors here: https://github.com/huggingface/transformers/issues/3387
# Change LR to 1e-3 to 1e-4
# 
ARGS_STR = """
--data_dir=/content/ \
--model_name_or_path=t5-base \
--learning_rate=3e-5 \
--train_batch_size=32 \
--output_dir=/content/output/ \
--do_train \
--n_gpu=1 \
--num_train_epochs 5 \
--max_source_length 80 \
"""
#
#--eval_batch_size=3 \
#--do_predict \

parser = argparse.ArgumentParser()
add_generic_args(parser, os.getcwd())
parser = T5Module.add_model_specific_args(parser, os.getcwd())
args = parser.parse_args(ARGS_STR.split())
trainer = main(args)

AttributeError: ignored